In [1]:
import time
import numpy as np
import tensorflow as tf
import utils

In [2]:
with open('D:/data/text8/text8') as f:
    text = f.read()

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [6]:
# subsampling
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word,count in word_counts.items()}
p_drop = {word: 1-np.sqrt(threshold / freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1-p_drop[word])]

In [8]:
def get_target(words, idx, window_size=5):
    R = np.random.randint(1,window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [9]:
def get_batches(words, batch_size, window_size=5):
    # 方便起见，弃掉最后那部分
    n_batches = len(words) // batch_size
    words = words[:n_batches*batch_size]
    
    for idx in range(0,len(words),batch_size):
        x,y = [],[]
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch,ii,window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x,y

In [10]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32,[None],name='inputs')
    labels = tf.placeholder(tf.int32,[None,None],name='labels')

In [11]:
# embedding
n_vocabs = len(int_to_vocab)
n_embedding = 300
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocabs,n_embedding),-1,1))
    embed = tf.nn.embedding_lookup(embedding,inputs)

In [12]:
# negative sampling
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocabs, n_embedding),stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocabs),name='softmax_bias')
    
    loss = tf.nn.sampled_softmax_loss(weights=softmax_w,
                                biases=softmax_b,
                                labels=labels,
                                inputs=embed,
                                num_sampled=n_sampled,
                                num_classes=n_vocabs)
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [14]:
epochs = 10
batch_size = 1000
window_size = 10

with tf.Session(graph=train_graph) as sess:
    loss = 0
    iteration = 1
    sess.run(tf.global_variables_initializer())
    
    for e in range(1,epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        for x,y in batches:
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100)
                      )
                loss = 0
            
            iteration += 1

Epoch 1/10 Iteration: 100 Avg. Training loss: 5.7547
Epoch 1/10 Iteration: 200 Avg. Training loss: 5.7264
Epoch 1/10 Iteration: 300 Avg. Training loss: 5.6096
Epoch 1/10 Iteration: 400 Avg. Training loss: 5.6931
Epoch 1/10 Iteration: 500 Avg. Training loss: 5.6150
Epoch 1/10 Iteration: 600 Avg. Training loss: 5.6071
Epoch 1/10 Iteration: 700 Avg. Training loss: 5.6814
Epoch 1/10 Iteration: 800 Avg. Training loss: 5.6269
Epoch 1/10 Iteration: 900 Avg. Training loss: 5.5714
Epoch 1/10 Iteration: 1000 Avg. Training loss: 5.5115


KeyboardInterrupt: 